In [ ]:
# this code is to perform the sequence clustering, to use this code
#1. prepare nextcalde clade info and aligned fasta file, can use nextclade CLI to do this easily
#2. install necessary package

In [4]:
import pandas as pd

# Step 1: Load the metadata file
metadata_file = "output_04_14/nextclade_04_14.tsv"
df = pd.read_csv(metadata_file, sep ="\t")

/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_84775/4274116231.py:5: DtypeWarning: Columns (28,50,51,52,53,54,55,56,57,58,59,61,75,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(metadata_file, sep ="\t")


In [ ]:
# Step 2: Drop lineages that occur only once, please remember to add this number for your signletons cluster
original_count = len(df)
df_filtered = df[df.groupby("Nextclade_pango")["Nextclade_pango"].transform("count") > 1]
filtered_count = len(df_filtered)
dropped_count = original_count - filtered_count
print(f"Number of sequences dropped (unique lineages): {dropped_count}")

# Step 3: Group by Lineage and extract sequence strain
lineage_to_seqnames = (
    df_filtered.groupby("Nextclade_pango")["seqName"]
    .apply(list)
    .to_dict()
)

Number of sequences dropped (unique lineages): 293


In [ ]:
# Step 4: Group sequenes from the aligned fasta file to individual file, I highly recommend using this approach as it is much easiler to run small file each time compare to super large file
from Bio import SeqIO
import os

input_fasta = "04_14_nextclade.aligned.fasta"  # replace with actual path

# Parse once, and store records by name for fast lookup
all_records = SeqIO.to_dict(SeqIO.parse(input_fasta, "fasta"))

# Make sure the output directory exists
os.makedirs("lineage_group_fasta", exist_ok=True)

# Then, write out grouped FASTAs
for lineage, names in lineage_to_seqnames.items():
    output_file = f"lineage_group_fasta/{lineage}_group.fasta"
    
    # Identify missing names
    missing_names = [name for name in names if name not in all_records]
    if missing_names:
        print(f"Warning: {len(missing_names)} sequences not found for lineage {lineage}")
        # Optionally, print the missing sequence names
        # print(f"Missing: {missing_names}")

    # Write only found sequences
    selected_records = [all_records[name] for name in names if name in all_records]
    
    if selected_records:
        with open(output_file, "w") as out_f:
            SeqIO.write(selected_records, out_f, "fasta")
    else:
        print(f"Warning: No sequences written for lineage {lineage} — all sequence names missing.")


In [ ]:
# Now, use pairsnp to caculate the distance matric of a given clade
import os
import numpy as np
import pandas as pd
from pairsnp import calculate_snp_matrix, calculate_distance_matrix

base_dir = "MN_data/lineage_group_fasta/"
test_one = True  # Set True to test just one group

# Track which clades are empty
empty_clades = []

# Loop through each grouped FASTA file
for filename in sorted(os.listdir(base_dir)):
    if filename.endswith("_group.fasta"):
        fasta_path = os.path.join(base_dir, filename)
        lineage_name = filename.replace("_group.fasta", "")

        try:
            print(f"Processing: {fasta_path}")

            # Run PairSNP analysis
            sparse_matrix, consensus, seq_names = calculate_snp_matrix(fasta_path)
            d = calculate_distance_matrix(sparse_matrix, consensus, "dist", False)

            # Save labeled distance matrix
            df = pd.DataFrame(d, index=seq_names, columns=seq_names)
            output_csv = os.path.join(base_dir, f"distance_matrix_group/{lineage_name}_snp_distance_matrix.csv")
            df.to_csv(output_csv)

        except ValueError as e:
            if "No sequences found" in str(e):
                print(f"⚠️ Skipping {lineage_name} — no sequences found.")
                empty_clades.append(lineage_name)
            else:
                raise  # Re-raise other unexpected errors

# Save the list of empty clades
if empty_clades:
    log_path = os.path.join(base_dir, "empty_clades_log.txt")
    with open(log_path, "w") as log_file:
        for clade in empty_clades:
            log_file.write(f"{clade}\n")

    print(f"\n📝 Empty clades saved to: {log_path}")


In [4]:
####this should be the accruate code, previous code doesn't consider when A = B, B=C, A not necessaily equals C
import os
import pandas as pd
from collections import defaultdict
import numpy as np

# === Set input directory containing *_snp_distance_matrix.csv files ===
input_dir = "lineage_group_fasta/distance_matrix_group/"

# === Set maximum SNP distance threshold for clustering ===
distance_threshold = 1  # Change this to 1, 2, or 3 as you like

# === Containers for outputs ===
cluster_summary = []
detailed_clusters = []

def find_strict_clusters(distance_df, threshold=0):
    """Find strict clusters where all members have pairwise distance <= threshold."""
    remaining = set(distance_df.index)
    clusters = []

    while remaining:
        # Start a new cluster with one sequence
        seed = remaining.pop()
        current_cluster = {seed}

        # Find initial candidates connected to the seed within threshold
        candidates = set(distance_df.columns[(distance_df.loc[seed] <= threshold)])

        # Iteratively expand the cluster
        expand = True
        while expand:
            expand = False
            for candidate in list(candidates):
                # Only consider candidate if still in remaining
                if candidate in remaining:
                    if all(distance_df.loc[candidate, member] <= threshold for member in current_cluster):
                        current_cluster.add(candidate)
                        remaining.remove(candidate)
                        expand = True
            # After expansion, re-calculate candidates
            candidates = {c for c in remaining if all(distance_df.loc[c, member] <= threshold for member in current_cluster)}

        clusters.append(current_cluster)

    return clusters


# === Loop through files ===
for file in sorted(os.listdir(input_dir)):
    if not file.endswith("_snp_distance_matrix.csv"):
        continue

    file_path = os.path.join(input_dir, file)
    lineage = file.replace("_snp_distance_matrix.csv", "")

    try:
        df = pd.read_csv(file_path, index_col=0)

        # Find strict clusters using the chosen threshold
        clusters = find_strict_clusters(df, threshold=distance_threshold)

        # Count clusters by size
        size_count = defaultdict(int)
        for cluster in clusters:
            size_count[len(cluster)] += 1

        # Add summary
        for size, count in size_count.items():
            cluster_summary.append({
                "Lineage": lineage,
                "Cluster_Size": size,
                "Count": count,
                "Threshold": distance_threshold
            })

        # Add detailed members
        for i, cluster in enumerate(clusters, 1):
            detailed_clusters.append({
                "Lineage": lineage,
                "Cluster_ID": f"{lineage}_StrictCluster_{i}",
                "Members": ";".join(sorted(cluster)),
                "Threshold": distance_threshold
            })

    except Exception as e:
        print(f"❌ Error processing {file}: {e}")

# === Convert to DataFrames ===
summary_df = pd.DataFrame(cluster_summary)
detailed_df = pd.DataFrame(detailed_clusters)

# === Save results ===
summary_output = os.path.join(input_dir, f"all_lineage_strict_cluster_summary_threshold_{distance_threshold}.csv")
detailed_output = os.path.join(input_dir, f"all_lineage_strict_cluster_membership_threshold_{distance_threshold}.csv")

summary_df.to_csv(summary_output, index=False)
detailed_df.to_csv(detailed_output, index=False)

print(f"✅ Finished. Strict clustering done with threshold {distance_threshold}.")


✅ Finished. Strict clustering done with threshold 1.


In [3]:
#additonal code for count number of each cluster
import pandas as pd

# === Load the cluster summary file ===
summary_path = "lineage_group_fasta/distance_matrix_group/all_lineage_strict_cluster_summary_threshold_0.csv"  # Update if needed
summary_df = pd.read_csv(summary_path)

# === Group by Cluster_Size and sum the counts ===
size_summary = summary_df.groupby("Cluster_Size")["Count"].sum().reset_index()

# === Save detailed counts for all cluster sizes ===
size_summary.to_csv("lineage_group_fasta/distance_matrix_group/cluster_size_summary_04_26_0nt.csv", index=False)

print("✅ Saved detailed cluster size summary to 'cluster_size_summary.csv'")



✅ Saved detailed cluster size summary to 'cluster_size_summary.csv'


In [7]:
#code to extract metadata to cluster from the large metadata file
# the code is slighlty redunant as the the seqName contains strain name from different source,but the point is to extract all the associated metadata of a given cluster for RR analysis
import pandas as pd
import os
import re

# === Load Data ===
membership_df = pd.read_csv("lineage_group_fasta/distance_matrix_group/summary_dataset_0nt/all_lineage_strict_cluster_membership_threshold_0.csv")
metadata_df = pd.read_csv("matched_metadata_final_drop_doplicate.csv")

# === Output folder ===
output_dir = "cluster_metadata_outputs_0nt"
os.makedirs(output_dir, exist_ok=True)

# === Helper function ===
def normalize_member_ids(members_raw):
    members_fixed = []
    for m in members_raw:
        match = re.search(r'hCoV-19/USA/([A-Z]{2}-[^/]+)', m)
        if match:
            extracted = match.group(1)
            print(f"✅ Match: {m} → {extracted}")
            members_fixed.append(extracted)
        else:
            print(f"❌ Failed match: {m}")
            members_fixed.append(None)
    return members_fixed

# === Skip clusters with only 1 member ===
membership_df["Member_Count"] = membership_df["Members"].apply(lambda x: len(x.split(";")))

for idx, row in membership_df.iterrows():
    cluster_id = row["Cluster_ID"]
    members_raw = row["Members"].split(";")

    if len(members_raw) <= 1:
        continue  # ✅ skip singleton clusters

    # Normalize member names
    members_fixed = normalize_member_ids(members_raw)

    if None in members_fixed:
        raise ValueError(f"❌ Cluster '{cluster_id}' has unrecognized member format: {members_raw}")

    # Match both dash and underscore forms
    members_fixed_all = set(members_fixed + [m.replace("-", "_") for m in members_fixed if m])
    metadata_matches = metadata_df[metadata_df["normalized_strain"].isin(members_fixed_all)].copy()
    metadata_matches["Cluster_ID"] = cluster_id

    # Raise error if any are missing
    matched_set = set(metadata_matches["normalized_strain"])
    missing = [m for m in members_fixed if m and m not in matched_set and m.replace("-", "_") not in matched_set]
    if missing:
        raise ValueError(f"❌ Cluster '{cluster_id}' has missing members: {missing}")

    # Save metadata for this cluster
    output_path = os.path.join(output_dir, f"{cluster_id}_metadata.tsv")
    metadata_matches.to_csv(output_path, sep='\t', index=False)
    print(f"✅ Saved: {cluster_id} ({len(metadata_matches)} records)")


/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_7410/491784292.py:9: DtypeWarning: Columns (24,25,27,28,30,31,32,33,35,36,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("matched_metadata_final_drop_doplicate.csv")


✅ Match: hCoV-19/USA/MN-CDC-IBX093742712808/2021 → MN-CDC-IBX093742712808
✅ Match: hCoV-19/USA/MN-CDC-IBX223667755029/2021 → MN-CDC-IBX223667755029
✅ Match: hCoV-19/USA/MN-CDC-IBX242356890105/2021 → MN-CDC-IBX242356890105
✅ Match: hCoV-19/USA/MN-CDC-IBX392990035837/2021 → MN-CDC-IBX392990035837
✅ Match: hCoV-19/USA/MN-CDC-IBX462753753308/2021 → MN-CDC-IBX462753753308
✅ Match: hCoV-19/USA/MN-CDC-IBX530358030653/2021 → MN-CDC-IBX530358030653
✅ Match: hCoV-19/USA/MN-CDC-IBX570182724829/2021 → MN-CDC-IBX570182724829
✅ Match: hCoV-19/USA/MN-CDC-IBX674975506073/2021 → MN-CDC-IBX674975506073
✅ Saved: AY.100_StrictCluster_2 (8 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX000460993918/2021 → MN-CDC-IBX000460993918
✅ Match: hCoV-19/USA/MN-CDC-IBX870060224085/2021 → MN-CDC-IBX870060224085
✅ Saved: AY.100_StrictCluster_4 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX084232867470/2021 → MN-CDC-IBX084232867470
✅ Match: hCoV-19/USA/MN-CDC-IBX595139060126/2021 → MN-CDC-IBX595139060126
✅ Saved: AY.100_StrictCl

✅ Saved: AY.100_StrictCluster_37 (15 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX216888881080/2021 → MN-CDC-IBX216888881080
✅ Match: hCoV-19/USA/MN-CDC-IBX529984483019/2021 → MN-CDC-IBX529984483019
✅ Saved: AY.100_StrictCluster_40 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX063894992260/2021 → MN-CDC-IBX063894992260
✅ Match: hCoV-19/USA/MN-CDC-IBX121420332322/2021 → MN-CDC-IBX121420332322
✅ Match: hCoV-19/USA/MN-CDC-IBX213615821275/2021 → MN-CDC-IBX213615821275
✅ Saved: AY.100_StrictCluster_43 (3 records)
✅ Match: hCoV-19/USA/MN-CDC-ASC210391909/2021 → MN-CDC-ASC210391909
✅ Match: hCoV-19/USA/MN-CDC-IBX203290235875/2021 → MN-CDC-IBX203290235875
✅ Saved: AY.100_StrictCluster_46 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX135606828339/2021 → MN-CDC-IBX135606828339
✅ Match: hCoV-19/USA/MN-CDC-IBX242924257175/2021 → MN-CDC-IBX242924257175
✅ Match: hCoV-19/USA/MN-CDC-IBX449411235466/2021 → MN-CDC-IBX449411235466
✅ Match: hCoV-19/USA/MN-CDC-IBX676127591463/2021 → MN-CDC-IBX676127591463
✅ Match: hC

✅ Saved: AY.100_StrictCluster_111 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-ASC210487982/2021 → MN-CDC-ASC210487982
✅ Match: hCoV-19/USA/MN-CDC-ASC210488262/2021 → MN-CDC-ASC210488262
✅ Match: hCoV-19/USA/MN-CDC-IBX656752823592/2021 → MN-CDC-IBX656752823592
✅ Match: hCoV-19/USA/MN-CDC-IBX900967420277/2021 → MN-CDC-IBX900967420277
✅ Saved: AY.100_StrictCluster_112 (4 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX139914157337/2021 → MN-CDC-IBX139914157337
✅ Match: hCoV-19/USA/MN-CDC-IBX562559871023/2021 → MN-CDC-IBX562559871023
✅ Match: hCoV-19/USA/MN-CDC-IBX625387292751/2021 → MN-CDC-IBX625387292751
✅ Match: hCoV-19/USA/MN-CDC-IBX674378342253/2021 → MN-CDC-IBX674378342253
✅ Saved: AY.100_StrictCluster_115 (4 records)
✅ Match: hCoV-19/USA/MN-CDC-ASC210287773/2021 → MN-CDC-ASC210287773
✅ Match: hCoV-19/USA/MN-CDC-IBX078824392830/2021 → MN-CDC-IBX078824392830
✅ Match: hCoV-19/USA/MN-CDC-IBX220778864057/2021 → MN-CDC-IBX220778864057
✅ Match: hCoV-19/USA/MN-CDC-IBX253225874577/2021 → MN-CDC-IBX25322

✅ Saved: AY.100_StrictCluster_199 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-ASC210407476/2021 → MN-CDC-ASC210407476
✅ Match: hCoV-19/USA/MN-CDC-IBX016713780314/2021 → MN-CDC-IBX016713780314
✅ Match: hCoV-19/USA/MN-CDC-IBX048822082885/2021 → MN-CDC-IBX048822082885
✅ Match: hCoV-19/USA/MN-CDC-IBX461468213362/2021 → MN-CDC-IBX461468213362
✅ Match: hCoV-19/USA/MN-CDC-IBX608347892044/2021 → MN-CDC-IBX608347892044
✅ Match: hCoV-19/USA/MN-CDC-IBX655148522134/2021 → MN-CDC-IBX655148522134
✅ Saved: AY.100_StrictCluster_200 (6 records)
✅ Match: hCoV-19/USA/MN-CDC-ASC210380518/2021 → MN-CDC-ASC210380518
✅ Match: hCoV-19/USA/MN-CDC-ASC210381488/2021 → MN-CDC-ASC210381488
✅ Saved: AY.100_StrictCluster_201 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-ASC210427065/2021 → MN-CDC-ASC210427065
✅ Match: hCoV-19/USA/MN-CDC-ASC210481672/2021 → MN-CDC-ASC210481672
✅ Match: hCoV-19/USA/MN-CDC-IBX876688283184/2021 → MN-CDC-IBX876688283184
✅ Saved: AY.100_StrictCluster_205 (3 records)
✅ Match: hCoV-19/USA/MN-CDC-IBX3

...

✅ Saved: XBB.1.5.91_StrictCluster_10 (2 records)
✅ Match: hCoV-19/USA/MN-MDH-35151/2023 → MN-MDH-35151
✅ Match: hCoV-19/USA/MN-MDH-35152/2023 → MN-MDH-35152
✅ Saved: XBB.1.5.91_StrictCluster_18 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00377/2023 → MN-CDC-VSX-A00377
✅ Match: hCoV-19/USA/MN-MDH-34807/2023 → MN-MDH-34807
✅ Saved: XBB.1.5.95_StrictCluster_2 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-2-6874942/2023 → MN-CDC-2-6874942
✅ Match: hCoV-19/USA/MN-CDC-2-6917715/2023 → MN-CDC-2-6917715
✅ Match: hCoV-19/USA/MN-MDH-33739/2023 → MN-MDH-33739
✅ Match: hCoV-19/USA/MN-MDH-34076/2023 → MN-MDH-34076
✅ Saved: XBB.1.5.95_StrictCluster_6 (4 records)
✅ Match: hCoV-19/USA/MN-MDH-34753/2023 → MN-MDH-34753
✅ Match: hCoV-19/USA/MN-MDH-34754/2023 → MN-MDH-34754
✅ Saved: XBB.1.5.95_StrictCluster_8 (2 records)
✅ Match: hCoV-19/USA/MN-MDH-33927/2023 → MN-MDH-33927
✅ Match: hCoV-19/USA/MN-MDH-33929/2023 → MN-MDH-33929
✅ Saved: XBB.1.5.95_StrictCluster_11 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-2-68749

✅ Saved: XBB.1.5_StrictCluster_85 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-VSX-A073060/2023 → MN-CDC-VSX-A073060
✅ Match: hCoV-19/USA/MN-MDH-33865/2023 → MN-MDH-33865
✅ Saved: XBB.1.5_StrictCluster_95 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-LC1024621/2023 → MN-CDC-LC1024621
✅ Match: hCoV-19/USA/MN-CDC-LC1027227/2023 → MN-CDC-LC1027227
✅ Match: hCoV-19/USA/MN-CDC-LC1028276/2023 → MN-CDC-LC1028276
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00290/2023 → MN-CDC-VSX-A00290
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00331/2023 → MN-CDC-VSX-A00331
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00396/2023 → MN-CDC-VSX-A00396
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00699/2023 → MN-CDC-VSX-A00699
✅ Match: hCoV-19/USA/MN-MDH-34851/2023 → MN-MDH-34851
✅ Match: hCoV-19/USA/MN-MDH-34854/2023 → MN-MDH-34854
✅ Saved: XBB.1.5_StrictCluster_96 (9 records)
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00353/2023 → MN-CDC-VSX-A00353
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00663/2023 → MN-CDC-VSX-A00663
✅ Saved: XBB.1.5_StrictCluster_97 (2 records)
✅ Match: hCoV-19/U

✅ Saved: XBB.1.5_StrictCluster_291 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-QDX46448829/2023 → MN-CDC-QDX46448829
✅ Match: hCoV-19/USA/MN-MDH-34820/2023 → MN-MDH-34820
✅ Saved: XBB.1.5_StrictCluster_302 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-LC1017563/2023 → MN-CDC-LC1017563
✅ Match: hCoV-19/USA/MN-MDH-34448/2023 → MN-MDH-34448
✅ Saved: XBB.1.5_StrictCluster_303 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-2-6874908/2023 → MN-CDC-2-6874908
✅ Match: hCoV-19/USA/MN-MDH-34255/2023 → MN-MDH-34255
✅ Match: hCoV-19/USA/MN-MDH-34827/2023 → MN-MDH-34827
✅ Saved: XBB.1.5_StrictCluster_306 (3 records)
✅ Match: hCoV-19/USA/MN-CDC-VSX-A00651/2023 → MN-CDC-VSX-A00651
✅ Match: hCoV-19/USA/MN-MDH-34849/2023 → MN-MDH-34849
✅ Match: hCoV-19/USA/MN-MDH-35025/2023 → MN-MDH-35025
✅ Saved: XBB.1.5_StrictCluster_310 (3 records)
✅ Match: hCoV-19/USA/MN-MDH-35332/2023 → MN-MDH-35332
✅ Match: hCoV-19/USA/MN-MDH-35339/2023 → MN-MDH-35339
✅ Saved: XBB.1.5_StrictCluster_312 (2 records)
✅ Match: hCoV-19/USA/MN-MDH-3535

✅ Saved: XBB.1.5_StrictCluster_861 (2 records)
✅ Match: hCoV-19/USA/MN-HLX-VSX-A04900/2023 → MN-HLX-VSX-A04900
✅ Match: hCoV-19/USA/MN-MDH-35876/2023 → MN-MDH-35876
✅ Saved: XBB.1.9.1_StrictCluster_10 (2 records)
✅ Match: hCoV-19/USA/MN-MDH-35530/2023 → MN-MDH-35530
✅ Match: hCoV-19/USA/MN-MDH-35531/2023 → MN-MDH-35531
✅ Saved: XBB.1.9.1_StrictCluster_11 (2 records)
✅ Match: hCoV-19/USA/MN-HLX-VSX-A05483/2023 → MN-HLX-VSX-A05483
✅ Match: hCoV-19/USA/MN-HLX-VSX-A05542/2023 → MN-HLX-VSX-A05542
✅ Saved: XBB.1.9.2_StrictCluster_2 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-QDX46449216/2023 → MN-CDC-QDX46449216
✅ Match: hCoV-19/USA/MN-MDH-34526/2023 → MN-MDH-34526
✅ Saved: XBB.1_StrictCluster_7 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-LC0998585/2023 → MN-CDC-LC0998585
✅ Match: hCoV-19/USA/MN-MDH-33958/2023 → MN-MDH-33958
✅ Saved: XBB.1_StrictCluster_21 (2 records)
✅ Match: hCoV-19/USA/MN-HLX-VSX-A04973/2023 → MN-HLX-VSX-A04973
✅ Match: hCoV-19/USA/MN-HLX-VSX-A04983/2023 → MN-HLX-VSX-A04983
✅ Mat

✅ Saved: XDA_StrictCluster_34 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-VSX-A14020/2023 → MN-CDC-VSX-A14020
✅ Match: hCoV-19/USA/MN-CDC-VSX-A14021/2023 → MN-CDC-VSX-A14021
✅ Match: hCoV-19/USA/MN-MDH-39273/2023 → MN-MDH-39273
✅ Saved: XDA_StrictCluster_37 (3 records)
✅ Match: hCoV-19/USA/MN-CDC-VSX-A14025/2023 → MN-CDC-VSX-A14025
✅ Match: hCoV-19/USA/MN-CDC-VSX-A17171/2023 → MN-CDC-VSX-A17171
✅ Saved: XDA_StrictCluster_50 (2 records)
✅ Match: hCoV-19/USA/MN-MDH-38634/2023 → MN-MDH-38634
✅ Match: hCoV-19/USA/MN-MDH-39660/2023 → MN-MDH-39660
✅ Saved: XDA_StrictCluster_66 (2 records)
✅ Match: hCoV-19/USA/MN-CDC-QDX36263184/2022 → MN-CDC-QDX36263184
✅ Match: hCoV-19/USA/MN-CDC-QDX36263185/2022 → MN-CDC-QDX36263185
✅ Match: hCoV-19/USA/MN-CDC-QDX36263338/2022 → MN-CDC-QDX36263338
✅ Match: hCoV-19/USA/MN-MDH-23478/2022 → MN-MDH-23478
✅ Saved: XQ_StrictCluster_2 (4 records)


In [24]:
#code for only keeping metadata row in the fastafile
import pandas as pd

# Load input files
metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")

# Step 1: Extract core ID from nextclade seqName (e.g., hCoV-19/USA/MN-CDC-XXX/2021)
nextclade_df['core_id'] = nextclade_df['seqName'].str.extract(r'hCoV-19/USA/([^/]+)', expand=False)

# Step 2: Replace underscores with dashes for consistency
nextclade_df['core_id'] = nextclade_df['core_id'].str.replace('_', '-', regex=False)
metadata_df['normalized_strain'] = metadata_df['strain'].str.replace('_', '-', regex=False)

# Step 3: Filter metadata to keep only matched strains
matched_metadata_df = metadata_df[metadata_df['normalized_strain'].isin(nextclade_df['core_id'])]

# Step 4: Save the matched metadata to a new file
matched_metadata_df.to_csv("matched_metadata_final.tsv", sep="\t", index=False)

# Step 5: Report number of matches
print(f"Number of matched metadata rows: {matched_metadata_df.shape[0]}")


/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/4142577797.py:5: DtypeWarning: Columns (24,25,27,28,30,31,32,33,35,36,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/4142577797.py:6: DtypeWarning: Columns (28,50,51,52,53,54,55,56,57,58,59,61,75,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")


Number of matched metadata rows: 82947


In [23]:
#checking for which row is present in the metadata

# Load metadata and Nextclade files
metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")

# Extract core ID from both files (e.g., MN-MDH-32441 or MN_UMGC_28146)
nextclade_df['core_id'] = nextclade_df['seqName'].str.extract(r'(MN[-_][A-Z]+[-_]\d+)')
metadata_df['core_id'] = metadata_df['covid_virus_name'].str.extract(r'(MN[-_][A-Z]+[-_]\d+)')

# Normalize both core_id columns by replacing underscores with dashes
nextclade_df['core_id'] = nextclade_df['core_id'].str.replace('_', '-', regex=False)
metadata_df['core_id'] = metadata_df['core_id'].str.replace('_', '-', regex=False)

# Filter to matched entries
matched_df = nextclade_df[nextclade_df['core_id'].isin(metadata_df['core_id'])]

# Print total number of matched entries
print("Number of matched sequences:", matched_df.shape[0])



/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/3735541180.py:4: DtypeWarning: Columns (24,25,27,28,30,31,32,33,35,36,37,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("2025-04-03_matched_MN_meta_toCHECK_v2.tsv", sep="\t")
/var/folders/9h/h1kjxm6j26ggb2qml3vbbxg80000gs/T/ipykernel_8813/3735541180.py:5: DtypeWarning: Columns (28,50,51,52,53,54,55,56,57,58,59,61,75,79,81,85) have mixed types. Specify dtype option on import or set low_memory=False.
  nextclade_df = pd.read_csv("output_04_14/nextclade_04_14.tsv", sep="\t")


Number of matched sequences: 82946
